**In this guide we will demonstrate how to use Nano AutoML to auto tune an machine learning model and handle the whole model development process easily.**

# Introduction
Nano provides hpo as an solution for fast and high-performance hyperparameter tuning. Users can import hano hpo following the standard tensorflow API without changing their own code. By setting the search space and running model.search(), the model will contain the best hyperparameters.

Nano hpo supports sequential, fuctional and custom keras API.


# Step0: Prepare Environment
We recommend using conda to prepare the environment.
conda create -n my_env python=3.7

conda activate my_env

pip install --pre --upgrade bigdl

# Step1: Init Nano AutoML
Let's start out by importing Nano AutoML and enable its tensorflow API globally by automl.hpo_config.enable_hpo_tf().

After importing, we will load MNIST as our data.

In [1]:
#to do: remove
import sys
sys.path.append("../../src/")

import bigdl.nano.automl as automl
automl.hpo_config.enable_hpo_tf()

from bigdl.nano.automl.tf.keras import Sequential
import bigdl.nano.automl.hpo.space as space
from bigdl.nano.tf.keras.layers import Dense, Flatten, Conv2D

from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf
from tensorflow import keras

N_TRAIN_EXAMPLES = 3000
N_VALID_EXAMPLES = 1000
CLASSES = 10

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
img_x, img_y = x_train.shape[1], x_train.shape[2]
x_train = x_train.reshape(-1, img_x, img_y,
                          1)[:N_TRAIN_EXAMPLES].astype("float32") / 255
x_test = x_test.reshape(-1, img_x, img_y,
                          1)[:N_VALID_EXAMPLES].astype("float32") / 255
y_train = y_train[:N_TRAIN_EXAMPLES]
y_test = y_test[:N_VALID_EXAMPLES]
input_shape = (img_x, img_y, 1)

../../src/bigdl/nano/tf/__init__.py:24: UserWarning: NANO_TF_INTER_OP not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("NANO_TF_INTER_OP not found the in os.environ, "
../../src/bigdl/nano/tf/__init__.py:30: UserWarning: OMP_NUM_THREADS not found the in os.environ, please run `source bigdl-nano-init`
  warnings.warn("OMP_NUM_THREADS not found the in os.environ, "


# Step2: Set search space
We now create our model function. 

Whenever you want to search a hyperparameter, you can simply replace it with a space and set your own searching space 


In [2]:
model = Sequential()

model.add(Conv2D(
    filters=space.Categorical(32, 64),
    kernel_size=space.Categorical(3, 5),
    strides=space.Categorical(1, 2),
    activation=space.Categorical("relu", "linear"),
    input_shape=input_shape))
model.add(Flatten())
model.add(Dense(CLASSES, activation="softmax"))

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.0001),
    metrics=["accuracy"]
)

2022-05-18 15:03:49.897047: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-18 15:03:49.897994: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 


# Step3: Apply searching
Apply hyperparameter searching by calling model.search(), and best hyperparameters will be saved in the model.

You can specify the number of trial, each trial will use a set of hyperparameters automatiacally suggusted by nano.hpo. 

You can set your target metric and whehter you want to maximize you minimize it.

In [3]:
%%time
model.search(
    n_trials=20,
    target_metric='accuracy',
    direction="maximize",
    x=x_train,
    y=y_train,
    batch_size=128,
    epochs=24,
    validation_split=0.2,
    verbose=False,
)

[I 2022-05-18 15:03:50,104] A new study created in memory with name: no-name-bf690e99-e883-4886-8dc5-7d42a3886663


Starting a new tuning
guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fd79403c190>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 2.2083 - accuracy: 0.4642 - val_loss: 2.1148 - val_accuracy: 0.6567 - 769ms/epoch - 40ms/step
Epoch 2/24
19/19 - 0s - loss: 2.0572 - accuracy: 0.6958 - val_loss: 1.9699 - val_accuracy: 0.7417 - 337ms/epoch - 18ms/step
Epoch 3/24
19/19 - 1s - loss: 1.9095 - accuracy: 0.7583 - val_loss: 1.8189 - val_accuracy: 0.7617 - 508ms/epoch - 27ms/step
Epoch 4/24
19/19 - 0s - loss: 1.7566 - accuracy: 0.7887 - val_loss: 1.6640 - val_accuracy: 0.7800 - 193ms/epoch - 10ms/step
Epoch 5/24
19/19 - 0s - loss: 1.6022 - accuracy: 0.7858 - val_loss: 1.5112 - val_accuracy: 0.7900 - 238ms/epoch - 13ms/step
Epoch 6/24
19/19 - 0s - loss: 1.4507 - accuracy: 0.8029 - val_loss: 1.3624 - val_accuracy: 0.7950 - 261ms/epoch - 14ms/step
Epoch 7/24
19/19 - 0s - loss: 1.3071 - a

[I 2022-05-18 15:03:57,912] Trial 0 finished with value: 0.8999999761581421 and parameters: {'activation▁choice': 0, 'filters▁choice': 1, 'kernel_size▁choice': 0, 'strides▁choice': 1}. Best is trial 0 with value: 0.8999999761581421.


guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fd764072950>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 2.2329 - accuracy: 0.2996 - val_loss: 2.1691 - val_accuracy: 0.4833 - 687ms/epoch - 36ms/step
Epoch 2/24
19/19 - 0s - loss: 2.1188 - accuracy: 0.5496 - val_loss: 2.0530 - val_accuracy: 0.6483 - 105ms/epoch - 6ms/step
Epoch 3/24
19/19 - 0s - loss: 1.9954 - accuracy: 0.6483 - val_loss: 1.9213 - val_accuracy: 0.7050 - 146ms/epoch - 8ms/step
Epoch 4/24
19/19 - 0s - loss: 1.8584 - accuracy: 0.7100 - val_loss: 1.7777 - val_accuracy: 0.7317 - 192ms/epoch - 10ms/step
Epoch 5/24
19/19 - 0s - loss: 1.7122 - accuracy: 0.7283 - val_loss: 1.6287 - val_accuracy: 0.7533 - 179ms/epoch - 9ms/step
Epoch 6/24
19/19 - 0s - loss: 1.5624 - accuracy: 0.7708 - val_loss: 1.4769 - val_accuracy: 0.7733 - 262ms/epoch - 14ms/step
Epoch 7/24
19/19 - 0s - loss: 1.4144 - accuracy: 0.7754 - val_los

[I 2022-05-18 15:04:02,726] Trial 1 finished with value: 0.89041668176651 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 1, 'strides▁choice': 1}. Best is trial 0 with value: 0.8999999761581421.


...

guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fd766e9e290>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 2.1152 - accuracy: 0.6137 - val_loss: 1.9503 - val_accuracy: 0.7700 - 1s/epoch - 56ms/step
Epoch 2/24
19/19 - 0s - loss: 1.8325 - accuracy: 0.7904 - val_loss: 1.6942 - val_accuracy: 0.7917 - 406ms/epoch - 21ms/step
Epoch 3/24
19/19 - 0s - loss: 1.5808 - accuracy: 0.8150 - val_loss: 1.4537 - val_accuracy: 0.8150 - 323ms/epoch - 17ms/step
Epoch 4/24
19/19 - 0s - loss: 1.3474 - accuracy: 0.8358 - val_loss: 1.2368 - val_accuracy: 0.8117 - 284ms/epoch - 15ms/step
Epoch 5/24
19/19 - 0s - loss: 1.1424 - accuracy: 0.8358 - val_loss: 1.0541 - val_accuracy: 0.8233 - 391ms/epoch - 21ms/step
Epoch 6/24
19/19 - 0s - loss: 0.9704 - accuracy: 0.8471 - val_loss: 0.9018 - val_accuracy: 0.8400 - 343ms/epoch - 18ms/step
Epoch 7/24
19/19 - 0s - loss: 0.8325 - accuracy: 0.8537 - val_los

[I 2022-05-18 15:07:37,848] Trial 18 finished with value: 0.9316666722297668 and parameters: {'activation▁choice': 0, 'filters▁choice': 0, 'kernel_size▁choice': 0, 'strides▁choice': 0}. Best is trial 2 with value: 0.9504166841506958.


guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fd767fa9950>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 - 1s - loss: 1.6927 - accuracy: 0.6775 - val_loss: 1.2791 - val_accuracy: 0.7867 - 863ms/epoch - 45ms/step
Epoch 2/24
19/19 - 0s - loss: 1.0838 - accuracy: 0.8112 - val_loss: 0.9073 - val_accuracy: 0.8200 - 424ms/epoch - 22ms/step
Epoch 3/24
19/19 - 0s - loss: 0.7962 - accuracy: 0.8404 - val_loss: 0.7026 - val_accuracy: 0.8383 - 456ms/epoch - 24ms/step
Epoch 4/24
19/19 - 1s - loss: 0.6283 - accuracy: 0.8612 - val_loss: 0.5829 - val_accuracy: 0.8567 - 760ms/epoch - 40ms/step
Epoch 5/24
19/19 - 1s - loss: 0.5251 - accuracy: 0.8717 - val_loss: 0.5133 - val_accuracy: 0.8550 - 724ms/epoch - 38ms/step
Epoch 6/24
19/19 - 1s - loss: 0.4553 - accuracy: 0.8879 - val_loss: 0.4533 - val_accuracy: 0.8783 - 575ms/epoch - 30ms/step
Epoch 7/24
19/19 - 0s - loss: 0.4057 - accuracy: 0.8954 - val_

[I 2022-05-18 15:07:51,030] Trial 19 finished with value: 0.9504166841506958 and parameters: {'activation▁choice': 1, 'filters▁choice': 1, 'kernel_size▁choice': 0, 'strides▁choice': 0}. Best is trial 2 with value: 0.9504166841506958.


CPU times: user 17min 48s, sys: 13min 14s, total: 31min 2s
Wall time: 4min


# Step 4: fit with the best hyperparameters
The best hyperparameters have been saved in the model. You can call model.fit() to train a model with the best hyperparameters.

In [4]:
history = model.fit(x_train, y_train,
                    batch_size=128, epochs=24, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

guoyao want to know compile args
() {'loss': 'sparse_categorical_crossentropy', 'optimizer': <keras.optimizer_v2.rmsprop.RMSprop object at 0x7fd7849e8d50>, 'metrics': ListWrapper(['accuracy'])}
Epoch 1/24
19/19 [==============================] - 1s 37ms/step - loss: 1.6927 - accuracy: 0.6775 - val_loss: 1.2791 - val_accuracy: 0.7867
Epoch 2/24
19/19 [==============================] - 1s 33ms/step - loss: 1.0838 - accuracy: 0.8112 - val_loss: 0.9073 - val_accuracy: 0.8200
Epoch 3/24
19/19 [==============================] - 0s 21ms/step - loss: 0.7962 - accuracy: 0.8404 - val_loss: 0.7026 - val_accuracy: 0.8383
Epoch 4/24
19/19 [==============================] - 0s 23ms/step - loss: 0.6283 - accuracy: 0.8612 - val_loss: 0.5829 - val_accuracy: 0.8567
Epoch 5/24
19/19 [==============================] - 0s 26ms/step - loss: 0.5251 - accuracy: 0.8717 - val_loss: 0.5133 - val_accuracy: 0.8550
Epoch 6/24
19/19 [==============================] - 0s 21ms/step - loss: 0.4553 - accuracy: 0.8879 - 

Check out the summary of the model. The model has already been built with the best hyperparameters found by nano hpo.

In [5]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 64)        640       
                                                                 
 flatten_1 (Flatten)         (None, 43264)             0         
                                                                 
 dense_1 (Dense)             (None, 10)                432650    
                                                                 
Total params: 433,290
Trainable params: 433,290
Non-trainable params: 0
_________________________________________________________________
None
